# Testing Model1

Testing recommender systems is less intuitive than testing other predictive models. There are many common metrics to use, which can be found [here in this useful Medium article](https://medium.com/swlh/rank-aware-recsys-evaluation-metrics-5191bba16832). The idea of these methods are all similar: generate a list of top ranked items to show the user, and your score is based on how many of the items you recommend are relevant to the user. The word "relevant" is not well-defined, but normally means that the user has interacted with this item in the past. 

In our case, our items are the restaurants and the interactions are orders. One issue with our data is that between 30-40% of our users have only ordered from a single restaurant, which makes the above methods tricky/impossible to apply.

Recall that our model takes in a user's past five (5) orders as inputs together with one restaurant R and tries to predict how likely the user will order from R next. We will evaluate our model as follows. Given a user's sequence of five restaurants, and given the target restaurant R which is next in the sequence, we use the model to generate a ranked list of $k=5$ more restaurants that it thinks is the most likely to be ordered from next. If R is among the $k$ generated restaurants, we add $+1$ to the score, otherwise $+0$. 

Since there are only $100$ restaurants to recommend from, it should be very feasible to simply score each one, sort the rankings, and slice the top 5 as recommendations. This is a very privillaged position we are in since many recommender systems are deployed in a context where there are millions of items to recommend from. In that case, one could perform clustering based on the customer and vendor embeddings before ranking within the clusters.  

Each user in the test set may possibly geenerate many length $5+1$ sequences, and we shall use all of these during the evaluation process. In the deployed version of the model, we could simply give recommendations based on the last $5$ orders made by the user.

Baselines:
1) Recommend the $5$ most popular restaurants.
2) Given a sequence of $5$ orders, $m$ of which are unique and not null, recommend those restaurants together with the $5-m$ most popular restaurants. 

We will define 'popularity' of a vendor to be equal to its number of orders in the training data. Note that the unordered set of top five most popular vendors would be the same if we changed the definition of popularity of a vendor to be equal to its number of unique customers (although their rankings are slightly different, see the last cell of "Munging.ipynb"). 

In [1]:
import pandas as pd
import pickle
import torch
from PreprocessingHelpers import CustomDataset
from torch.utils.data import Dataset, DataLoader
from Models.Models import Model1


## Load Test Data

In [2]:
with open("ProcessedData/test_sequences_padded_dataset.pkl", "rb") as file:
    test_sequences_padded_dataset = pickle.load(file)

test_loader = DataLoader(test_sequences_padded_dataset, batch_size=1)
num_trials = test_sequences_padded_dataset.vendor.shape[0]

In [3]:
with open("ProcessedData/vendors_tensor.pkl", "rb") as file:
    vendors_tensor = pickle.load(file)

model = Model1(vendors=vendors_tensor)

In [4]:
with open("ProcessedData/popular_vendors.pkl", "rb") as file:
    popular_vendors = pickle.load(file)

## Define Scoring for Model & Baselines

In [5]:
popular_vendors.head(10)

id
28    3237
25    2717
14    2681
19    2453
18    2184
15    2159
75    1377
21    1274
36    1145
6     1078
Name: num_orders, dtype: int64

In [6]:
k_most_popular = popular_vendors[:5].index.tolist()
k_most_popular

[28, 25, 14, 19, 18]

In [7]:
def baseline1_scoring(target:int, k_most_popular:list):
    if target in k_most_popular:
        return 1
    else:
        return 0

In [8]:
def baseline2_scoring(seq:list, target:int, k_most_popular:list):
    seq = list(set(seq))    # remove duplicates
    try:
        seq.remove(0)       # remove null-token
    except ValueError:
        pass
    m = len(seq)
    seq = seq + k_most_popular[:-m]
    if target in seq:
        return 1
    else:
        return 0

In [9]:
def model1_scoring(seq:torch.tensor, target:int, model:Model1, k:int=5):
    seq = seq.view(1, -1)
    y = torch.ones([100, 1], dtype=torch.long)      # 100 vendors
    seq_dupe = y @ seq                              # 100 x 5 matrix
    v_ids = torch.arange(start=1, end=101, dtype=torch.long)
    rankings = model.forward(c_seq=seq_dupe, v_id=v_ids).view(-1)    
    top_k = torch.topk(rankings, k)[1][:5]          # Essentially argmax for top k
    if target in top_k:
        return 1
    else:
        return 0

In [10]:
# Score baselines
print("SCORING\n=======================================================")
baseline1_score = 0
baseline2_score = 0
for i, (c_seq, v_id) in enumerate(test_loader):
    target = v_id.item()
    c_seq_list = c_seq.view(-1).tolist()
    baseline1_score += baseline1_scoring(target=target, k_most_popular=k_most_popular)
    baseline2_score += baseline2_scoring(seq=c_seq_list, target=target, k_most_popular=k_most_popular)
print(f'Baseline1:\t{baseline1_score} / {num_trials} = {baseline1_score*100/num_trials:.2f}%')
print(f'Baseline2:\t{baseline2_score} / {num_trials} = {baseline2_score*100/num_trials:.2f}%')

# Score models at different epochs
for epoch in range(0, 101, 10):
    PATH = "Models/model1_epoch"+str(epoch)+".pt"
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    model1_score = 0
    for i, (c_seq, v_id) in enumerate(test_loader):
        model.eval()
        target = v_id.item()
        c_seq_list = c_seq.tolist()
        with torch.no_grad():
            model1_score += model1_scoring(seq=c_seq, target=target, model=model)
    print(f'Model1_{epoch}:\t{model1_score} / {num_trials} = {model1_score*100/num_trials:.2f}%')
print("Done!")

SCORING
Baseline1:	9233 / 47677 = 19.37%
Baseline2:	22134 / 47677 = 46.42%
Model1_0:	3546 / 47677 = 7.44%
Model1_10:	1468 / 47677 = 3.08%
Model1_20:	1618 / 47677 = 3.39%
Model1_30:	3475 / 47677 = 7.29%
Model1_40:	2923 / 47677 = 6.13%
Model1_50:	3887 / 47677 = 8.15%
Model1_60:	2067 / 47677 = 4.34%
Model1_70:	3629 / 47677 = 7.61%
Model1_80:	2056 / 47677 = 4.31%
Model1_90:	1605 / 47677 = 3.37%
Model1_100:	2426 / 47677 = 5.09%
Done!
